<a href="https://colab.research.google.com/github/naaz-verma/Graph_Visualization/blob/main/Graph_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/data.csv")

In [ ]:
data.shape

(2894, 7)

In [ ]:
data.head(5)

,title,volume,authors,year,pages,link,code
0,Joint Estimation and Inference for Data Integr...,Volume 23,"['Subhabrata Majumdar', 'George Michailidis']",2022,53,https://www.jmlr.org/papers/volume23/18-131/18...,https://github.com/GeorgeMichailidis/JMMLE_code
1,Debiased Distributed Learning for Sparse Part...,Volume 23,"['Shaogao Lv', 'Heng Lian']",2022,32,https://www.jmlr.org/papers/volume23/18-467/18...,NaN
2,Recovering shared structure from multiple netw...,Volume 23,"['Keith Levin', 'Asad Lodhia', 'Elizaveta Levi...",2022,48,https://www.jmlr.org/papers/volume23/19-1056/1...,NaN
3,Exploiting locality in high-dimensional Factor...,Volume 23,"['Lorenzo Rimella', 'Nick Whiteley']",2022,34,https://www.jmlr.org/papers/volume23/19-267/19...,https://github.com/LorenzoRimella/GraphFilter-...
4,Empirical Risk Minimization under Random Censo...,Volume 23,"['Guillaume Ausset', 'Stephan ClÃ©menÃ§on', 'F...",2022,59,https://www.jmlr.org/papers/volume23/19-450/19...,NaN


In [ ]:
import re
for i in data['title']:
  i = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", i)
  i=re.sub(r"\s"," ",i)

In [ ]:
for i in range(len(data['title'])):
  data['title'][i]=data['title'][i].lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
data.head()

,title,volume,authors,year,pages,link,code
0,joint estimation and inference for data integr...,Volume 23,"['Subhabrata Majumdar', 'George Michailidis']",2022,53,https://www.jmlr.org/papers/volume23/18-131/18...,https://github.com/GeorgeMichailidis/JMMLE_code
1,debiased distributed learning for sparse part...,Volume 23,"['Shaogao Lv', 'Heng Lian']",2022,32,https://www.jmlr.org/papers/volume23/18-467/18...,NaN
2,recovering shared structure from multiple netw...,Volume 23,"['Keith Levin', 'Asad Lodhia', 'Elizaveta Levi...",2022,48,https://www.jmlr.org/papers/volume23/19-1056/1...,NaN
3,exploiting locality in high-dimensional factor...,Volume 23,"['Lorenzo Rimella', 'Nick Whiteley']",2022,34,https://www.jmlr.org/papers/volume23/19-267/19...,https://github.com/LorenzoRimella/GraphFilter-...
4,empirical risk minimization under random censo...,Volume 23,"['Guillaume Ausset', 'Stephan ClÃ©menÃ§on', 'F...",2022,59,https://www.jmlr.org/papers/volume23/19-450/19...,NaN


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()
x = v.fit_transform(data['title'][:600])


In [ ]:
print(x.toarray())
print(x.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(600, 1475)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(x)

In [ ]:
import networkx as nx
G = nx.to_networkx_graph(similarity_matrix)

In [ ]:
import plotly.graph_objects as go
def create_node_trace(G):
    # collect node information from G to plot
    node_x = []
    node_y = []
    node_text = []
    node_color = []

    for i, node in enumerate(G.nodes(data=True)):
        # get node x,y position and store
        x, y = node[1]['pos']
        node_x.append(x)
        node_y.append(y)

        node_text.append(node[1]['text'])
        node_color.append(node[1]['color'])

    # create node trace (i.e., scatter plot)
    # make it invisible by default
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=False,
            color=node_color,
            size=16,
            line_width=0.5,
        ),
        text=node_text,
        visible=False
    )

    return node_trace

In [ ]:
def create_edge_trace(G):
    # collect edges information from G to plot
    edge_weight = []
    edge_text = []
    edge_pos = []
    edge_color = []
    
    for edge in G.edges(data=True):
        
        # edge is line connecting two points
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_pos.append([[x0, x1, None], [y0, y1, None]])
        
        # edge line color when drawn
        edge_color.append("black")

    # there is a trace for each edge
    edge_traces = []
    for i in range(len(edge_pos)):
        
        # edge line width
        line_width = 1

        # is scatter because it is line connecting two points
        trace = go.Scatter(
            x=edge_pos[i][0], y=edge_pos[i][1],
            line=dict(width=line_width, color=edge_color[i]),
            mode='lines',
            visible=False
        )
        edge_traces.append(trace)

    return edge_traces

In [ ]:
def filter_similarity_matrix_at_step(square_matrix, step_value):
    # copy matrix
    aux = square_matrix.copy()
    
    # set as NaN all values equal to or below threshold value
    aux[aux <= step_value] = np.nan
    
    # return filtered matrix
    return aux

In [ ]:
def get_interactive_slider_similarity_graph(square_matrix, slider_values, node_text=None, yaxisrange=None, xaxisrange=None):
    
    # Create figure with plotly
    fig = go.Figure()

    slider_dict = {}
    total_n_traces = 0
    
    for i, step_value in enumerate(slider_values):

        aux = filter_similarity_matrix_at_step(square_matrix, step_value)

       
        G = nx.to_networkx_graph(aux)
        
        G.remove_edges_from([(a, b) for a, b, attrs in G.edges(data=True) if np.isnan(attrs["weight"])])

        node_pos = nx.nx_pydot.graphviz_layout(G)

        for node in G.nodes(data=True):
            
          
            node[1]['pos'] = node_pos[node[0]]

       
            node[1]['color'] = "orange"

            if node_text is not None:
                node[1]['text'] = node_text[node[0]]
            else:
                node[1]['text'] = ""

        edge_traces = create_edge_trace(G)
        
        node_trace = create_node_trace(G) 

        slider_dict[step_value] = edge_traces + [node_trace]
        
       
        total_n_traces += len(slider_dict[step_value])

        if i == 0:
            for trace in slider_dict[step_value]:
                # make visible
                trace.visible = True

         
    steps = []
    for step_value in slider_values:
        
        n_traces_before_adding_new = len(fig.data)
        
        fig.add_traces(slider_dict[step_value])

        step = dict(
            method="update",
        
            args=[{"visible": [False] * total_n_traces}],
         
            label=str(round(step_value, 3)),
        )

        n_traces_for_step_value = len(slider_dict[step_value])
        for i in range(n_traces_before_adding_new, n_traces_before_adding_new + n_traces_for_step_value):
            step["args"][0]["visible"][i] = True
        
     
        steps.append(step)

    slider = [dict(
        active=0,
        steps=steps
    )]

    fig.update_layout(
        sliders=slider,
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(range=xaxisrange, showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(range=yaxisrange, showgrid=False, zeroline=False, showticklabels=False),
        width=700, height=700,
    )

    return fig

In [ ]:
titles=[i for i in data['title'][:600]]

In [ ]:
# define slider steps (i.e., threshold values)
slider_steps = np.arange(0.2, 0.85, 0.05)
    
# get the slider figure
fig = get_interactive_slider_similarity_graph(
    similarity_matrix,
    slider_steps,
    node_text = titles
)

# plot it
fig.show()

For particular words

In [ ]:
word='deep learning'

In [ ]:
working_data = pd.DataFrame(columns = ['title'])
for i in range(len(data['title'])):
  if word in data['title'][i]:
    working_data.loc[i]=data['title'][i]

In [ ]:
working_data.shape

(11, 1)

In [ ]:
working_data

,title
7,deep learning in target space
190,an inertial newton algorithm for deep learning
255,on admm in deep learning: convergence and satu...
296,dig: a turnkey library for diving into graph d...
297,sparsity in deep learning: pruning and growth ...
369,gluoncv and gluonnlp: deep learning in compute...
598,traces of class/cross-class structure pervade ...
807,deep hidden physics models: deep learning of n...
1029,deep learning the ising model near criticality
1031,maximum principle based algorithms for deep le...


In [ ]:
titles_search=[i for i in working_data['title']]

In [ ]:
v_search = TfidfVectorizer()
x_search = v_search.fit_transform(working_data['title'])
similarity_matrix_search = cosine_similarity(x_search)
G_search = nx.to_networkx_graph(similarity_matrix_search)
titles_search=[i for i in working_data['title']]
slider_steps = np.arange(0.1, 0.85, 0.05)
    
# get the slider figure
fig = get_interactive_slider_similarity_graph(
    similarity_matrix_search,
    slider_steps,
    node_text = titles_search
)

# plot it
fig.show()